# Import Libraries

In [1]:
import copy
import utils
import torch
import constants
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from env import Env
from agent import Agent
from torchsummary import summary
from torch.distributions import Normal, Categorical

[MAX_POSSIBLE_DIST]: 0.4428093360578569


# Initialise Environment

In [2]:
#initialise environment
min_x, max_x =  -0.110 - 0.150,   -0.110 + 0.150
min_y, max_y =   0.560 - 0.125,    0.560 + 0.125
min_z, max_z =               0,              0.4 

workspace_lim = np.asarray([[min_x, max_x], 
                            [min_y, max_y],
                            [min_z, max_z]])

print(f"workspace space: \n{workspace_lim}")

obj_dir = 'objects/blocks/'
N_obj   = 5

env = Env(obj_dir, N_obj, workspace_lim, cluttered_mode= False, is_debug = False)

workspace space: 
[[-0.26   0.04 ]
 [ 0.435  0.685]
 [ 0.     0.4  ]]


# Initialise Agent

In [3]:
agent = Agent(env, 
              max_memory_size = 25000, 
              max_memory_size_rl = 1,
              max_memory_size_hld = 25000,
              is_debug = False, 
              N_batch = 512, 
              N_batch_hld = 512, 
              lr = 1e-4, 
              hld_lr = 1e-3,
              tau = 0.01,
              tau_hld = 0.01,
              max_action_taken = 50,
              max_result_window = 500,
              max_result_window_hld = 250,
              max_result_window_eval = 100,
              max_stage1_episode = 200,
              save_all_exp_interval = 100)

device: cuda
[SUCCESS] initialise environment
[SUCCESS] initialise networks


# Interact

In [5]:
agent.interact(max_episode = 2, 
               hld_mode    = constants.HLD_MODE,
               is_eval     = True)

[SUCCESS] load agent data
[LOAD MODEL] load hld-net best model
[SUCCESS] load hld-net model
[LOAD MODEL] load grasp best model
[SUCCESS] load grasp model
[LOAD MODEL] load push best model
[SUCCESS] load push model
[SUCCESS] restart environment
['4.obj']
item 0: shape_0, path index: 0, pose: [-0.08724793718341867, 0.599690291524857, 0.025, 3.4422345736404267, 2.099208367307221, 3.2908213593755447]
item 1: shape_1, path index: 0, pose: [-0.0019918833635384076, 0.43734625233505237, 0.025, 5.760872454365773, 5.1663968069707265, 4.715848909788138]
item 2: shape_2, path index: 0, pose: [-0.010493487842370669, 0.6180259308130298, 0.025, 1.773832338990826, 3.638192588768103, 2.412682034757909]
item 3: shape_3, path index: 0, pose: [-0.15438029068907377, 0.6793513843958716, 0.025, 2.4851838416581296, 3.142920321704619, 4.216256454729387]
item 4: shape_4, path index: 0, pose: [-0.09336792212023307, 0.5064812161459447, 0.025, 5.481150234513644, 4.993804890172228, 0.7515409291047017]
[SUCCESS] res

In [5]:
# agent.hld_mode = constants.GRASP_ONLY
# agent.is_eval  = True
agent.load_agent_data()

In [ ]:
CR_rate = np.sum(agent.CR_eval)/agent.max_result_window_eval
print(f"completion rate (eval): {CR_rate*100.}%")

plt.plot(agent.CR_eval)

In [ ]:
AGS_rate = np.sum(np.array(agent.AGS_eval)*np.array(agent.CR_eval))/(np.array(agent.CR_eval) > 0).sum()
print(f"AGS rate (eval): {AGS_rate*100.}%")

plt.plot(agent.AGS_eval)

In [ ]:
ATC_rate = np.sum(np.array(agent.ATC_eval)*np.array(agent.CR_eval))/(np.array(agent.CR_eval) > 0).sum()
print(f"ATC rate (eval): {ATC_rate}")

plt.plot(agent.ATC_eval)
plt.ylim([0, 25])

In [ ]:
(np.array(agent.CR_eval) > 0).sum()